In [1]:
import numpy as np
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image
from skimage.measure import compare_ssim,compare_psnr
np.random.seed(73)

In [2]:
def residual_block(input_tense):
    tmp1 = tf.nn.relu(tf.layers.batch_normalization(
        tf.layers.conv2d(input_tense,64,[3,3],[1,1], padding='SAME')))
    tmp2 = tf.nn.relu(tf.layers.batch_normalization(
        tf.layers.conv2d(tmp1,16,[2,2],[1,1], padding='SAME')))
    tmp3 = tf.layers.batch_normalization(
        tf.layers.conv2d(tmp2,1,[1,1],[1,1], padding='SAME'))
    tmp4 = tmp3 + input_tense
    return tmp4

In [3]:
from xy_inception_tf_model import inception_module,inception_model

def recon_NN(comp_in, NC, block_size, stride_size, reuse=False):
    with tf.variable_scope('Decomp_NN_884_fix_code_RB_SF_JPEG_85', reuse=reuse):
    
        with tf.name_scope('Pre_dcomp'):
            pdcomp = residual_block(comp_in)
            print(pdcomp)

        with tf.name_scope('Dcompress_Conv'):
            dcomp = tf.layers.conv2d_transpose(pdcomp,NC,[block_size,block_size],[stride_size,stride_size], padding='VALID')
            print(dcomp)


        with tf.name_scope('inception_model'):
            outputs=inception_model(dcomp,8)
        
        return outputs

In [4]:
block_size = 8
stride_size = 8

nx=128
ny=240
ncomp=4

sc_int=23.
Q_scale=1.

NC = 1

f_code=h5py.File('./code_884_1x16_mix_4bit.h5','r')
code=f_code['data'][:]
code=code.astype(np.float32)
print("code shape: " + str(code.shape))

code shape: (8, 8, 1, 4)


In [5]:
with tf.name_scope('Compress_Conv_f32'):
    compf32 = tf.placeholder(tf.float32,shape=[None,nx,ny,ncomp])
print(compf32)

Tensor("Compress_Conv_f32/Placeholder:0", shape=(?, 128, 240, 4), dtype=float32)


In [6]:
recon=recon_NN(compf32, NC, block_size, stride_size)
print("\nrecon: " + str(recon))

Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/Pre_dcomp/add:0", shape=(?, 128, 240, 4), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/Dcompress_Conv/conv2d_transpose/BiasAdd:0", shape=(?, 1024, 1920, 1), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/conv2d/BiasAdd:0", shape=(?, 1024, 1920, 64), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/batch_normalization/FusedBatchNorm:0", shape=(?, 1024, 1920, 64), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/conv2d_1/BiasAdd:0", shape=(?, 1024, 1920, 32), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/batch_normalization_1/FusedBatchNorm:0", shape=(?, 1024, 1920, 32), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/LeakyRelu/Maximum:0", shape=(?, 1024, 1920, 96), dtype=float32)
Tensor("Decomp_NN_884_fix_code_RB_SF_JPEG_85/inception_model/inception_modules_0/conv2d/BiasAdd:0", shape=(?, 1

In [7]:
trained_model = './model_884_4bit_fix_code_RB_SF_JPEG_85/model_200000.ckpt-200001'
saver = tf.train.Saver()

In [8]:
def run_recon(data_in):
    with tf.Session() as sess:
        saver.restore(sess,trained_model)
        dip=sess.run(recon,feed_dict={compf32:data_in})
            
    return dip

In [9]:
def read_COMPINT_JPEG(j_name):
    o4=np.zeros([1, int(nx*2), int(ny*2), ncomp])
    for k in range(ncomp):
        inname = j_name + "_C_" + str(k) + ".jpeg"
        #print("  Input: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
        
    o4 = (o4-128.)/(sc_int*Q_scale)
    o4 = o4.astype(np.float32)
        
    return o4

In [10]:
def read_raw_data(r_name):
    xs=0; ys=0
    xe=xs+2048; ye=ys+3840
    pop=np.fromfile(r_name,dtype=np.dtype('i2'))
    pop=pop[0:3864*2174]
    
    pop=np.reshape(pop,(2174,3864))
    pop=np.rot90(np.rot90(pop))  #For 1, 2, comment out if running 3
    
    pop=pop[xs:xe,ys:ye]
    #pop=np.rot90(np.rot90(pop))  #For 3, comment out if running 1, 2
    
    pop=pop/(np.power(2,14)-1.)
    
    return pop

In [11]:
def write_hdf5(x,filename):
    with h5py.File(filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)

In [12]:
cxs_s={}; cys_s={};

cxs_s[0]=0; cys_s[0]=0
cxs_s[1]=nx; cys_s[1]=0
cxs_s[2]=0; cys_s[2]=ny
cxs_s[3]=nx; cys_s[3]=ny

for i in range(4):
    print(str(i) + " cxs= " + str(cxs_s[i]) + " , cys= " + str(cys_s[i]))

0 cxs= 0 , cys= 0
1 cxs= 128 , cys= 0
2 cxs= 0 , cys= 240
3 cxs= 128 , cys= 240


In [13]:
xs_s={}; ys_s={};
Nx=1024; Ny=1920

xs_s[0]=0; ys_s[0]=0
xs_s[1]=Nx; ys_s[1]=0
xs_s[2]=0; ys_s[2]=Ny
xs_s[3]=Nx; ys_s[3]=Ny

for i in range(4):
    print(str(i) + " xs= " + str(xs_s[i]) + " , ys= " + str(ys_s[i]))

0 xs= 0 , ys= 0
1 xs= 1024 , ys= 0
2 xs= 0 , ys= 1920
3 xs= 1024 , ys= 1920


In [14]:
def big_frame_recon(data_in):
    Dips={}
    for i in range(4):
        cxs=cxs_s[i]; cxe=cxs+nx
        cys=cys_s[i]; cye=cys+ny
        c_f=np.zeros([1,nx,ny,ncomp])
        c_f[0,:,:,:]=data_in[0,cxs:cxe,cys:cye,:]
        Dips[i]=run_recon(c_f)
        
    shape_o=[1, int(Nx*2), int(Ny*2), 1]
    Dip=np.zeros(shape_o)
    for i in range(4):
        xs=xs_s[i]; xe=xs+Nx
        ys=ys_s[i]; ye=ys+Ny
        Dip[0,xs:xe,ys:ye,0]=Dips[i][0,:,:,0]
        
    return Dip

In [15]:
fileName='../Example_raw_bayer_data/1.raw'

jpname = './tmp_COMPINT'

In [16]:
t_raw = read_raw_data(fileName)

In [17]:
print("  t_raw shape= " + str(t_raw.shape) + " , max= " + str(np.max(t_raw)) 
        + " , min= " + str(np.min(t_raw)) + " , mean= " + str(np.mean(t_raw)))

  t_raw shape= (2048, 3840) , max= 0.9999389611182323 , min= 0.05273759384728072 , mean= 0.27559552881423693


In [18]:
t_jpeg = read_COMPINT_JPEG(jpname)

In [19]:
print("  t_jpeg shape= " + str(t_jpeg.shape) + " , max= " + str(np.max(t_jpeg)) 
        + " , min= " + str(np.min(t_jpeg)) + " , mean= " + str(np.mean(t_jpeg)))

  t_jpeg shape= (1, 256, 480, 4) , max= 3.3043478 , min= -2.5217392 , mean= 0.5308596


In [20]:
r_raw = big_frame_recon(t_jpeg)

INFO:tensorflow:Restoring parameters from ./model_884_4bit_fix_code_RB_SF_JPEG_85/model_200000.ckpt-200001
INFO:tensorflow:Restoring parameters from ./model_884_4bit_fix_code_RB_SF_JPEG_85/model_200000.ckpt-200001
INFO:tensorflow:Restoring parameters from ./model_884_4bit_fix_code_RB_SF_JPEG_85/model_200000.ckpt-200001
INFO:tensorflow:Restoring parameters from ./model_884_4bit_fix_code_RB_SF_JPEG_85/model_200000.ckpt-200001


In [21]:
mse=np.mean(np.square(r_raw[0,:,:,0]-t_raw))
print("mse= " + str(mse) + "\n")

mse= 0.00023510102263140197



In [22]:
output=(r_raw[0,:,:,0]*255).astype('uint8')
label=(t_raw*255).astype('uint8')

psnr =compare_psnr(output, label)
ssim =compare_ssim(output, label, multichannel=False)
print(str(i) + " , PSNR= " + str(psnr) + " , SSIM= " + str(ssim))

3 , PSNR= 36.233121428682445 , SSIM= 0.9744634718619178


In [23]:
write_hdf5(r_raw,'image_884_rec_dbl_SF_JPEG85.h5')